
#Workflow:

##Target:
finetune LLama model version: on dataset: specific on area: dialog summary

##Dataset:
huggingface

##Base model:
setup Libraries\
Download LLama model\
Base model Config setup(quantization setting step is here)\
import pre-trained model\
setup tokenizer\



##Data preprocess:
  1: formatting text and summary to "prompt+input:text + output:summary"for model\
  2: tokenization, transfet text to token_ids(with attention_mask)
  3: use base model to generate 'raw' summary for feeling.

##Finetune process:
  1:trainning setup:\
    1.1: Set up QLora Moudle(actually LoraConfig, base parameters W parameters NF4 setting + quantization is in earlier base model Config)
    1.2: Set up peft_trainer
    1.3: .train()

  2:check training process:\
    trainable model parameters\
    all model parameters\
    percentage of trainable model parameters

  3: train model\

##post-training:

1.upload Peft_model from based model + Lora tuend model

2. test its ability of generating dialog summary. Compare its results with base model's.

##Important comments:

1.I don't have good local GPU, and that's main reason I put this project on colab. One biggest debug issue I constantly have is error of torch.device(None). 我在peft_model.train(cuda)时触发 torch.device(None) 的报错，debug去掉了peft_model.to('cuda') 以及原模型 device—map=‘cuda’ 改成‘auto’ 后没问题了。可能是某些子模块或者参数（比如lora参数）没有放到GPU上，引发这个 “device index None” 的错误。
然后改成device_map='auto'，Accelerate就会动为模型的不同子模块分配设备，避免设备不一致带来的问题。
也可以参考这个回复https://stackoverflow.com/questions/78008119/huggingface-transformer-train-function-throwing-device-received-an-invalid-com?utm_source=chatgpt.com

2.If you want to run this file, pls change hugging face access token with your own, and before this step you need to apply premission from
Meta Llama 3 on hugging face. The access token shown in this file is changed by randomly generated one, and you will get login error with it.

3.On github you can see modified version of this project. That evaluate generated summaries in more details.

4.关于checkpointing 主要作用在训练的 前向／反向传播中的激活保存与梯度重计算；而 use_cache 主要作用在推理阶段的 Key/Value 缓存复用，这俩不是一个阶段的事情，开启Key/Value 缓存复用不会加速训练速度。另外我看到有个帖子讨论这件事，https://discuss.huggingface.co/t/why-is-use-cache-incompatible-with-gradient-checkpointing/18811，我的理解是在训练中，通常是一次处理完整序列，模型并不逐 token 生成、累积 past key values。此时 use_cache=True 会让模型保存／返回 past_key_values，但这些缓存可能会干扰后续梯度计算或被视为“状态”而非激活，从而与 checkpointing 的重新计算逻辑冲突。

In [ ]:
%pip  install accelerate peft bitsandbytes transformers trl pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.3 MB/s eta 0:00:00


In [ ]:
import os
from huggingface_hub import login
from datasets import load_dataset
from transformers import (
AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig,
TrainingArguments,
pipeline,
logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from tqdm import tqdm
import torch
import time
import pandas as pd
import numpy as np
from pprint import pprint

Next cell is upload LLama model name on huggingface, and dialog summary dataset "neil-code/dialogsum-test". Print first 10 rows to check data content.

In [ ]:
HF_TOKEN = "hf_wengwiornewogvjajwet"  # enter huggingface account token "hf_xxx"; 如果留空且模型受限会报权限错误 (p.s. in open version it's a fake token)
if HF_TOKEN:
  login(HF_TOKEN)

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

dataset_name = "neil-code/dialogsum-test"
dataset = load_dataset(dataset_name)

pprint(dataset['train'][:3], width=100, compact=False) # pprint is useful to see output of cell in a block rather than in a line


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

{'dialogue': ["#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n"
              '#Person2#: I found it would be a good idea to get a check-up.\n'
              "#Person1#: Yes, well, you haven't had one for 5 years. You should have one every "
              'year.\n'
              '#Person2#: I know. I figure as long as there is nothing wrong, why go see the '
              'doctor?\n'
              '#Person1#: Well, the best way to avoid serious illnesses is to find out about them '
              'early. So try to come at least once a year for your own good.\n'
              '#Person2#: Ok.\n'
              '#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, '
              'please. Do you smoke, Mr. Smith?\n'
              '#Person2#: Yes.\n'
              '#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. '
              'You really should quit.\n'
              "#Person2#: I've tried hundreds of time

In next cell, we build a function to format data into instruction data. instruction data is for prompt use.

Notice that this function works on every single 'context combo'(id,dialog,summary,topic) and return a (text) added into the 'context combo'.

In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    :param sample: input data
    """
    INTRO_BLURB = """Instruct: Below is an instruction that describes a task.
Write a response that appropriately completes the request."""
    INSTRUCTION_KEY = "Input: Please Summarize the below conversation."
    RESPONSE_KEY = "Output:"
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    parts = [part for part in [blurb, instruction, input_context, response] if
    part]
    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt
    return sample

print(create_prompt_formats(dataset['train'][0])['text'])


Instruct: Below is an instruction that describes a task.
Write a response that appropriately completes the request.

Input: Please Summarize the below conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medicatio

In next cell we need to build QLoRA setting. This is because we will quantlize base weight matrix from bf16 to 4-bit, and decoder it on the fly of each training iteration. So Let's start.

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=True,
)

upload base model(here we use meta-llama/Llama-3.1-8B). We also check improvement of the size of model with our quantization_config(NF4 for base model weight parameter matrix)

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

original_model = AutoModelForCausalLM.from_pretrained(
model_id,
dtype=compute_dtype, #`torch_dtype` is deprecated! Use `dtype` instead!
device_map="auto",
quantization_config=quant_config
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

add tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False,
  trust_remote_code=True,
  padding_side="left",add_eos_token=True,add_bos_token=True)#left padding to promise in generation process we put non-EOS token to right side

tokenizer.pad_token_id = tokenizer.eos_token_id #llama has special padding token, just to play safe, so we use eos token to be pading token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Tese our base model before fine-tune: inference：
1: build gen() function to create template
2: use tokenizer to decoder and


In [ ]:
%%time
eval_tokenizer = tokenizer

def gen(model,p, maxlen=200, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen,do_sample=sample,num_return_sequences=1,temperature=0.5,num_beams=1,top_p=0.99,
    ).to('cpu') #multi options to generate: beam search, top k, top p. We choose top p here.##generate expects named tensors (input_ids, attention_mask, …) on the same device as the model.
    #Using .to("cuda") + ** cleanly moves and passes them.
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)


from transformers import set_seed
seed = 100
set_seed(seed) #fix the random choosing happened in gen() below
index = 1
prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']
formatted_prompt = f"Instruct: Summarize the following conversation.\nInput: {prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
dash_line = '-'.join('' for x in range(200))
print(res[0])
print(dash_line)
output = res[0].split('Output:\n')[1]
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - 0-SHOT:\n{output}') # My feedback after reading the results: baseline is true summary, and 0-shot is more like keep writing the story from prompt.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruct: Summarize the following conversation.
Input: #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It waste

Main stage:

We want to finetune base model now.

The first step is data preprocess. We already created new dataset[train]['text'], which combines input:\n text + output:\n summary together.
Now we use tokenizer we setup earlier, and transfer text data to token ids. (embedding from tokens to vectors is inside llama transformer model, so we don't need to do this step)

It's good data for model to learn now.

Next few cells is for data preprocess.

In [ ]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings",
                           "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(batch["text"],max_length=max_length,truncation=True,) # truncation(means cut) and padding(means add) generate tokens of each text paragraph with same length.

In [ ]:
from functools import partial
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed,
dataset):
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#function in .map() acts on dataset sample by sample
    _preprocessing_function = partial(preprocess_batch, max_length=max_length,
    tokenizer=tokenizer)#fixed input max—length and tokenizer with preprocess—batch function because too many input，I may make mistake later
    dataset = dataset.map(
    _preprocessing_function,
    batched=True,
    remove_columns=['id', 'topic', 'dialogue', 'summary'],
    )#function in .map() acts on dataset sample by sample

    # filter data to drop sample that longer than max_length,
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) <=
    max_length)
    # Shuffle data with fixed seed to keep result fixed
    dataset = dataset.shuffle(seed=seed)
    print("Preprocessing dataset done.")
    return dataset

In [ ]:
# apply those tools and setups above to our dataset
max_length = get_max_length(original_model)
print(max_length)
train_dataset = preprocess_dataset(tokenizer, max_length,seed,
dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed,
dataset['validation']) #validation dataset

Found max lenth: 8192
8192
Preprocessing dataset...


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Preprocessing dataset done.
Preprocessing dataset...


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

Preprocessing dataset done.


In second step, we build a finetune model. There are two parts: first part is setup of Lora and second is finetune model. Notice that we basicly just work on parameters of LoraConfig and transformers.trainer. We don't need to build these two moudle, but it's necessary to know the concepts and structure of these two.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
r=16, #Rank I use T4 GPU, so r=32 is too 'OOM' for me.
lora_alpha=16,
target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
bias="none",
lora_dropout=0.01, # Conventional
task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()
# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)
peft_model = get_peft_model(original_model, config)

In [ ]:
output_dir = '/content/peft-dialogue-summary-training/final-checkpoint'
import transformers
from transformers import TrainingArguments

peft_training_args = TrainingArguments(
output_dir = output_dir,
warmup_steps=1,
per_device_train_batch_size=1,
gradient_accumulation_steps=1,
max_steps=512, # 'OOM'problem, I have to reduce maz_steps
learning_rate=2e-4,
optim="paged_adamw_8bit",
logging_steps=100,
logging_dir="/content/logs",
save_strategy="steps",
save_steps=100,
eval_strategy="steps", #should replace evaluation_strategy="steps" with eval_strategy="steps". The argument name was changed in newer versions of the transformers library.
eval_steps=100,
do_eval=True,
gradient_checkpointing=True,
report_to="none",
overwrite_output_dir = True,
group_by_length=True,
#save_total_limit=2,             # 控制checkpoint数量，省空间
)

peft_model.config.use_cache = False

# Explicitly move the model to the GPU before initializing the trainer
#peft_model.to('cuda')#为什么不要 peft_model.to('cuda')

#对 4bit/8bit（bitsandbytes）+ device_map="auto" 的模型，手动 .to('cuda') 容易让部分量化层仍留在 CPU，导致 Accelerate 在比较设备时拿到 None（CPU 的 index），从而触发 torch.device(None) 的报错。
#让 from_pretrained(device_map="auto") 和 Trainer/Accelerate 统一管理设备放置最稳妥。


peft_trainer = transformers.Trainer(
model=peft_model,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
args=peft_training_args,
data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,
mlm=False),
)

Train peft model, and check how many parameters trained in training process.

Free memory after training because we only care trained new model.

In [ ]:
peft_trainer.train()
# OutOfMemoryError OOM 多半是“序列太长 + CE上 fp32 中间张量占内存”。首选降低 max_length，并开启 fp16/bf16。
#训练先关评估，能跑通后再恢复。
#不手动 .to("cuda")，把设备交给 device_map="auto"。
#必要时把 LoRA rank 降到 8/16。









Step,Training Loss,Validation Loss
100,1.476600,1.436036
200,1.361000,1.420452
300,1.349800,1.412919
400,1.364300,1.406743
500,1.331300,1.402241


TrainOutput(global_step=512, training_loss=1.3813901655375957, metrics={'train_runtime': 866.0768, 'train_samples_per_second': 0.591, 'train_steps_per_second': 0.591, 'total_flos': 5977928817254400.0, 'train_loss': 1.3813901655375957, 'epoch': 0.256128064032016})

Since we encountered issues with the standard `Trainer`, let's try using the `SFTTrainer` from the `trl` library, which is designed for supervised fine-tuning tasks like this and might handle the integration with QLoRA and BitsAndBytes more smoothly.

In [ ]:
'''This is a backup training plan if the standard Trainer above fails.
from trl import SFTTrainer

# Reusing the previously defined model, tokenizer, datasets, and training arguments

sft_trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=config, # Use the LoRA config here
    dataset_text_field="text", # Specify the column containing the formatted text
    max_seq_length=max_length,
    tokenizer=tokenizer,
    args=peft_training_args,
    packing=False,
)

# Start training with the SFTTrainer
sft_trainer.train()'''

'This is a backup training plan if the standard Trainer above fails.\nfrom trl import SFTTrainer\n\n# Reusing the previously defined model, tokenizer, datasets, and training arguments\n\nsft_trainer = SFTTrainer(\n    model=peft_model,\n    train_dataset=train_dataset,\n    eval_dataset=eval_dataset,\n    peft_config=config, # Use the LoRA config here\n    dataset_text_field="text", # Specify the column containing the formatted text\n    max_seq_length=max_length,\n    tokenizer=tokenizer,\n    args=peft_training_args,\n    packing=False,\n)\n\n# Start training with the SFTTrainer\nsft_trainer.train()'

After training, you can evaluate the model and save the trained adapter weights.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\n \
all model parameters: {all_model_params}\n \
percentage of trainable model parameters: \
{100 * trainable_model_params / all_model_params:.2f}%"

# original_model is deleted before this function is called, so this will raise an error.
# I'll print the trainable parameters before deleting the original model.
print_number_of_trainable_model_parameters(peft_model)




'trainable model parameters: 13631488\n all model parameters: 4554231808\n percentage of trainable model parameters: 0.30%'

In [ ]:
# Free memory（optional，I use colab pro and choose 80G ram, which is more than enough）
'''del original_model # original_model is not defined in this cell, so this will raise an error.
del peft_trainer
torch.cuda.empty_cache()'''

'del original_model # original_model is not defined in this cell, so this will raise an error.\ndel peft_trainer\ntorch.cuda.empty_cache()'

Now added trained Lora parameters \delta W to base model parameters W, and check by generating summary.

In [ ]:
#import base model again if uncomment Free memory step.
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=compute_dtype,
bnb_4bit_use_double_quant=True,
)
model_path = "meta-llama/Meta-Llama-3-8B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
model_path,
dtype=compute_dtype,#`torch_dtype` is deprecated! Use `dtype` instead!
device_map={"": 0},
quantization_config=quant_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#import tokenizer again if uncomment Free memory step.
eval_tokenizer = AutoTokenizer.from_pretrained(model_path, add_bos_token=True,
trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
# Merge parameters from Lora and base model
from peft import PeftModel
ft_model = PeftModel.from_pretrained(
base_model,
"/content/peft-dialogue-summary-training/final-checkpoint/checkpoint-500",
torch_dtype=torch.float16,
is_trainable=False
) #作用：把“已经训练好的 LoRA 适配器权重”从磁盘（或Hub）加载到一个基座模型上，返回一个可用的 PeftModel。
#注意：from_pretrained 是“挂载适配器”，并没有把 ΔW 合并进基座权重。若你想得到“已融合”的单一模型，需要再调用 merge_and_unload()

In [ ]:
%%time
index = 0
prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']
formatted_prompt = f"Instruct: Summarize the following conversation.\nInput:{prompt}\nOutput:\n"
res = gen(ft_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]
dash_line = '-'.join('' for x in range(200))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'PEFT MODEL GENERATION:\n{output}')

# It's clear to see that fine-tuned ft_model has a summary close to human baseline summary from dataset.
# Compared with base model, ft_model has slightly better ability of dialog summary!
# I add 'scores' to evaluate how close the generated summary is to human baseline summary. Please check it on github

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
Input:#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, 